In [1]:
import chromadb
#https://github.com/chroma-core/chroma/blob/main/chromadb/experimental/density_relevance.ipynb

client =  chromadb.PersistentClient('./chroma_docs')
col_raw = client.get_collection('documents_raw')
#col_raw = client.get_collection('documents_summary')

data = col_raw.get()

print(data.keys())
len(data['documents'])

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data', 'included'])


3180

In [2]:
results = col_raw.query(query_texts="what is graph rag?",n_results=5)
print(results.keys())
results['documents']


dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data', 'included'])


[['9.4 GraphRAG in Industry\n\nIn this section, we mainly focus on industrial GraphRAG systems. These systems are characterized by their reliance on industrial graph database systems or their focus on large-scale graph data, details of which are as follows.\n\n• GraphRAG (by Microsoft)10: The system uses LLMs to construct entity-based knowledge graphs and pre-generate community summaries of related entity groups, which enables the capture of both local and global relationships within a document collection, thereby enhancing Query-Focused Summarization (QFS) task [32]. The project can also utilize open-source RAG toolkits for rapid implementation, such as LlamaIndex11, LangChain12, etc.\n\n• GraphRAG (by NebulaGraph)13: The project is the first industrial GraphRAG system, which is developed by NebulaGraph Corporation. The project integrates LLMs into the NebulaGraph database, which aims to deliver more intelligent and precise search results.\n\n• GraphRAG (by Antgroup)14: The framework 

In [54]:
import litellm
import os
from json_repair import loads
"""
1. Questions About Relationships Between Entities
Example: "What is the relationship between Elon Musk and Tesla?"
2. Complex Queries Involving Multiple Entities
Example: "Which employees work in the marketing department of Company X, and what projects are they currently managing?"
5. Contextual Understanding
Example: "What impact did the new regulations have on the energy sector in Germany?"
10. Role-based Queries
Example: "What projects has the project manager of the 'Apollo' initiative worked on in the last year?"
11. Dynamic Updates and Changes
Example: "What recent partnerships has Company C formed, and how do they relate to its previous collaborations?"
12. Hierarchical Queries
Example: "Who are the direct reports of the CEO of Company D, and what are their responsibilities?"
13. Understanding Influence and Networks
Example: "Who are the key influencers in the climate change advocacy network, and how are they connected?"
14. Temporal Progressions and Trends
Example: "How has the production capacity of Company E changed over the last decade?"

"""
question_examples ="""
3. Path Queries
Example: "What are the key events leading to the acquisition of Company Y by Company Z?"
4. Aggregate Data Queries
Example: "What are the total sales figures for all subsidiaries of Company A in the last fiscal year?"
6. Temporal Relationships
Example: "What changes occurred in the management team of Company A over the last five years?"
7. Multi-hop Reasoning
Example: "Which authors have written books that were adapted into movies starring Tom Hanks?"
8. Comparative Analysis with Context
Example: "How does the market share of Company A compare to Company B in the automotive industry?"
9. Causal Relationships
Example: "What were the causes of the economic downturn in 2008 according to the data?"
15. Event Correlations
Example: "What global events have influenced the stock prices of tech companies in the past year?"
16. Collaborative Contributions
Example: "Which researchers have collaborated on multiple projects in renewable energy, and what are their contributions?"
"""

create_base_question_prompt ="""
From the following piece of TEXT create questions with answers. below some question examples are provided:
** Examples **
{question_examples}

** Examples **
** TEXT **
{doc}

** TEXT **
Your response is in JSON format with these fileds.
{{
Question: a question from the TEXT 
Answer: an answer to the question
Number: the question number start from 0
}}

You have to create 10 questions.
"""

evolve_question_prompt= """
You are given a BASE_Question and its answer. You have to create short different questions from the BASE_Question by changing it.
BASE_Question: {question}
Answer: {answer}
Your response is in JSON array with these fileds.
'''json
Reasoning: an array of questions focuing on improving of the reasoning of the topic.
Deepening: an array of questions each goes deeper into the topic.
Constraint: an array of questions that has some constraints.
Complicate:  an array of questions that complicate the input.
Evolving:  an array of questions that each is completly different from the BASE_Question but it must be in the same topic.
Abstract: an array of questions that focus on abstract concepts.
'''

each array has 5 element and questions are not even remotely similar to each other.
"""


answer_prompt="""Answer to this question based on the following text in a JSON format like:
{{"Answer": a short answer to the question based on the text or inferred from the text
"Refrences": from which parts of the text Answer is extracted i.e [Data: Source]
"Answerable": true if you can extract the answer from the **Text** or false if you can not.
}}

** Text **
{context_data}

** Text **

Output:
"""

#Difference: A number between 0-10 indicating diffence of this set of questions to the previous sets.
#

In [4]:
import pandas as pd
communities_path  = '/home/cip/ce/ix05ogym/Majid/LLM/GraphRag/elec_graph/output/create_final_community_reports.parquet'
communities_reports = pd.read_parquet(communities_path)
communities_reports_content = communities_reports['full_content']
communities_reports_content

0     # Soldering and Electronics Assembly\n\nThis c...
1     # Flux and Solderability\n\nThis community rev...
2     # Assembly and its Associated Faults\n\nThis c...
3     # Component Assembly and Manufacturing\n\nThis...
4     # Electrical Connections and Circuits\n\nThis ...
                            ...                        
68    # BS 4584 and Printed Circuit Board Laminates\...
69    # Solderability and Electronics Assembly\n\nTh...
70    # Printed Wiring Boards and Related Processes\...
71    # Metal and Solder Joints in Electronics\n\nTh...
72    # Soldering and Electronics Assembly\n\nThis c...
Name: full_content, Length: 73, dtype: object

In [65]:
communities_reports[communities_reports['level']==1]

,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
15,13,# Soldering and Electronics Assembly\n\nThis c...,1,7.0,Soldering and Electronics Assembly,The impact severity rating is high due to the ...,This community revolves around the process of ...,[{'explanation': 'Soldering is a fundamental p...,"{\n ""title"": ""Soldering and Electronics Ass...",48732510-a76e-44db-b604-8b5791b2420e
16,14,# Soldering and its Associated Phenomena\n\nTh...,1,7.0,Soldering and its Associated Phenomena,The impact severity rating is high due to the ...,This community revolves around the process of ...,[{'explanation': 'Wetting is a crucial step in...,"{\n ""title"": ""Soldering and its Associated ...",93af2a96-2fbe-42ed-8cc2-e41d522dff32
17,15,# Electronics Assemblies and Soldering Process...,1,7.0,Electronics Assemblies and Soldering Processes,The impact severity rating is high due to the ...,This community revolves around the creation of...,[{'explanation': 'Electronics assemblies are t...,"{\n ""title"": ""Electronics Assemblies and So...",3f7fe7a5-dc93-4ec3-ab39-53d39685258c
18,16,# Soldering and Contamination\n\nThis communit...,1,3.0,Soldering and Contamination,The impact severity rating is low due to the l...,This community focuses on the process of solde...,[{'explanation': 'Contamination can significan...,"{\n ""title"": ""Soldering and Contamination"",...",dc103fc9-c87a-45e3-ad12-5636b258aa14
19,17,# Solder Paste and its Applications\n\nThis co...,1,3.0,Solder Paste and its Applications,The impact severity rating is low due to the t...,"This community revolves around solder paste, a...",[{'explanation': 'Solder paste is the central ...,"{\n ""title"": ""Solder Paste and its Applicat...",dee157ed-4292-4f0f-b71b-aef90e77de3d
20,18,# Soldering and Further Reading\n\nThis commun...,1,2.0,Soldering and Further Reading,The impact severity rating is low due to the l...,This community revolves around the concept of ...,[{'explanation': 'Soldering is the central top...,"{\n ""title"": ""Soldering and Further Reading...",38764982-ef47-4325-8631-5d4a7a64cc51
21,19,# Glossary and Terms in Soldering\n\nThis comm...,1,2.0,Glossary and Terms in Soldering,The impact severity rating is low due to the l...,This community focuses on the relationship bet...,[{'explanation': 'A glossary is a key resource...,"{\n ""title"": ""Glossary and Terms in Solderi...",9fa83409-8e13-4f7d-8081-39b9c6a4abd2
22,20,# Hand Soldering and Soldering Iron\n\nThis co...,1,2.0,Hand Soldering and Soldering Iron,The impact severity rating is low due to the l...,This community revolves around the process of ...,[{'explanation': 'Hand soldering is a fundamen...,"{\n ""title"": ""Hand Soldering and Soldering ...",42d499dc-1e4e-4eb9-83c4-ffb8dd4389c3
23,21,# ITRI and Lead-Free Soldering\n\nThis communi...,1,3.0,ITRI and Lead-Free Soldering,The impact severity rating is low due to the l...,This community centers around the research and...,[{'explanation': 'ITRI is a key player in the ...,"{\n ""title"": ""ITRI and Lead-Free Soldering""...",34c5307b-d566-4409-9a44-86d1872d5bab
24,22,# Soldering Problems and Solutions\n\nThis com...,1,2.0,Soldering Problems and Solutions,The impact severity rating is low because the ...,This community focuses on the relationship bet...,[{'explanation': 'Problems are a key entity in...,"{\n ""title"": ""Soldering Problems and Soluti...",d2bcf490-da80-445d-b7fd-4dad004465b1


In [58]:
from  utils import electronic_collection,completion,answer_to_quesion

def get_not_found_questions(base_questions):
    not_found = []
    for row in base_questions:
        question = row['Question']
        number = row['Number']
        print(number , question)
        res = answer_to_quesion(electronic_collection,question,answer_prompt,True)
        print(res)
        try:
            if res['Answerable'] == False:
                res['Question']=question
                res['Graph_Answer']=row['Answer']
                not_found.append(res)
        except :
            continue
    return not_found


In [66]:

#data = electronic_collection.get(limit=1)
#doc = data['documents'][1]
mylist =[]
for i in range(15,30):
    base_questions = completion(create_base_question_prompt.format(doc=communities_reports_content[i] ,question_examples=question_examples),)
    display(base_questions)

    not_found = get_not_found_questions(base_questions)
    mylist.append(not_found)
    display(not_found)

[{'Question': 'What are the key steps involved in soldering and what is its primary purpose in electronics assembly?',
  'Answer': 'Soldering involves joining metal parts by melting solder, which then cools and solidifies. This process is essential for connecting electronic components to printed circuit boards, forming the foundation of electronic assemblies.',
  'Number': 0},
 {'Question': 'How does temperature and cleanliness affect the soldering process?',
  'Answer': 'The solder needs to be heated to the correct temperature for it to melt and flow properly. Additionally, clean surfaces on the components and the printed circuit board are crucial for a strong bond. Contamination can lead to poor-quality joints.',
  'Number': 1},
 {'Question': 'What are the safety considerations involved in soldering and how are they addressed?',
  'Answer': 'Soldering involves working with hot materials and potentially hazardous chemicals. Safety is ensured through careful product design, adherence t

0 What are the key steps involved in soldering and what is its primary purpose in electronics assembly?
{'Answer': 'Soldering, in essence, is the process of joining two metallic surfaces together by melting solder, an alloy typically composed of tin and lead, between them. The solder solidifies, creating a strong and conductive bond.  The primary purpose of soldering in electronics assembly is to create both a mechanical and electrical connection between components, ensuring they are securely held together while also allowing electrical current to flow through the circuit.', 'Refrences': '[Data: Source unit_text soldering.pdf_8]', 'Answerable': True}
1 How does temperature and cleanliness affect the soldering process?
{'Answer': 'Temperature affects the soldering process by determining the rate of melting and solidification of the solder, as well as the reaction rate of the flux.  Cleanliness is essential for successful soldering as any contaminants can prevent the solder from properly

[{'Answer': "The text focuses on the technical aspects of soldering, not its social or political implications.  Therefore, it doesn't directly discuss how soldering might contribute to peaceful pursuits. However, we can infer that by enabling the creation of electronic devices, soldering indirectly contributes to peaceful activities like communication, education, and research.",
  'Refrences': [],
  'Answerable': False,
  'Question': 'How might soldering contribute to peaceful pursuits?',
  'Graph_Answer': "The text suggests that soldering could potentially contribute to peaceful endeavors, but it doesn't elaborate on specific applications."}]

[{'Question': 'What is soldering and how does it work?',
  'Answer': 'Soldering is a process where molten solder is used to join metal surfaces. It involves wetting the surfaces with the solder, which then solidifies to create a strong bond.',
  'Number': 0},
 {'Question': 'What are the key entities involved in the soldering process, and how do they interact?',
  'Answer': 'The key entities are wetting, oxides, dewetting, intermetallic compounds, surface tension, non-wetting, and contaminants. They interact through complex relationships influencing the success or failure of soldering.',
  'Number': 1},
 {'Question': 'Why is wetting a crucial step in soldering?',
  'Answer': 'Wetting ensures the molten solder makes direct contact with the metal surfaces, forming a strong alloy and bond. It requires clean surfaces free of contaminants to allow the solder to touch the metal atoms.',
  'Number': 2},
 {'Question': 'How do oxides affect the soldering process?',
  'Answer': 'Oxides form on me

0 What is soldering and how does it work?
{'Answer': 'Soldering is a process used in the electronics industry to bond components together, forming one or more electrical connections.  It works by melting solder, an alloy typically of tin and lead, and allowing it to flow between the components being joined, creating a strong mechanical and electrical bond.', 'Refrences': '[Data: Source unit_text soldering.pdf_8]', 'Answerable': True}
1 What are the key entities involved in the soldering process, and how do they interact?
{'Answer': 'The key entities in soldering are the solder, the components to be joined, and the flux. The solder is a metal alloy that melts at a relatively low temperature, allowing it to flow and bond the components together.  The flux, a chemical paste or liquid, helps to clean the surfaces of the components and solder, preventing oxidation and ensuring proper wetting. The process involves applying heat to the joint, melting the solder, and allowing it to flow into t

...

0 What are the fundamental units that make up electronic circuits?
{'Answer': 'Electronic circuits are fundamentally built using components like resistors, capacitors, inductors, diodes, transistors, and integrated circuits.', 'Refrences': '[Data: Source unit_text soldering.pdf_11]', 'Answerable': True}
1 What is the purpose of printed circuit boards (PCBs) in electronic assemblies?
{'Answer': 'Printed circuit boards (PCBs) are used to hold and connect electronic components in an organized and structured way within electronic assemblies.', 'Refrences': '[Data: Source unit_text soldering.pdf_11]', 'Answerable': True}
2 What are the different types of soldering processes used in the assembly of electronic products?
{'Answer': 'There are two main categories of soldering processes: Component/Solder (CS) processes and Solder/Component (SC) processes. CS processes involve placing components onto the board and then applying solder, while SC processes involve applying solder before placing com

[]

[{'Question': 'What are the main soldering processes discussed in the text, and which one is considered the most modern?',
  'Answer': 'The text discusses Wave Soldering, Dip Soldering, and Drag Soldering. Wave soldering is considered the most modern and widely used process.',
  'Number': 0},
 {'Question': 'What are the key differences between Wave Soldering and Dip Soldering in terms of contact time and temperature?',
  'Answer': 'Wave soldering requires a shorter contact time and a lower solder temperature compared to Dip soldering.',
  'Number': 1},
 {'Question': 'What is the purpose of flux in soldering, and how does it relate to wetting?',
  'Answer': "Flux cleans the surfaces of components and the printed circuit board before soldering, while wetting refers to the solder's ability to spread and adhere to the surfaces. Both are crucial for successful soldering.",
  'Number': 2},
 {'Question': 'What is CS soldering, and how is it different from SC soldering?',
  'Answer': 'CS solde

0 What are the main soldering processes discussed in the text, and which one is considered the most modern?
{'Answer': 'The text discusses two main soldering processes: CS (component/solder) processes and SC (solder/component) processes. CS processes involve placing components on a board and then applying solder, while SC processes apply solder before placing components. The most modern soldering process discussed in the text is SC soldering, specifically in the context of mass soldering techniques. The text mentions that a growing number of manufacturers are transitioning to SC processes, using solder paste to apply solder before placing components, and then applying heat to create the joints.', 'Refrences': '[Data: Source unit_text soldering.pdf_22], [Data: Source unit_text soldering.pdf_24]', 'Answerable': True}
1 What are the key differences between Wave Soldering and Dip Soldering in terms of contact time and temperature?
{'Answer': 'Wave soldering offers significantly shorter con

[]

In [6]:
#evol = completion(evolve_question_prompt.format(question = base_questions[1]['Question'] , answer = base_questions[0]['Answer']))
#display(evol)

In [62]:
not_found = get_not_found_questions(base_questions)
not_found

TypeError: string indices must be integers, not 'str'

In [78]:
question = "What concepts are connected to wave soldering?"
question = "what are top 5 important topics in soldering? write it with a number from 1 to 5"
question = "what are 5 least important topics in soldering? write it with a number from 1 to 5"
question = "does cleaning come before solidification?"
question = "what cause soldification quality decrease?"
question = "if I have a poor solder can I still reach a high quality soldering? How?"
question = "give me a comperhensive summary of solidification"

res = answer_to_quesion(electronic_collection,question,answer_prompt,True)
res

{'Answer': 'Solidification is the process where a substance transitions from a liquid state to a solid state. This occurs when the molecules within the liquid lose energy, slow down, and become more closely packed, forming a regular, crystalline structure. Solidification is often accompanied by a decrease in volume and a release of latent heat. The specific temperature at which solidification occurs is called the freezing point, and it is a unique property of each substance. The process can be influenced by factors like pressure, impurities, and the presence of nucleation sites. Solidification plays a crucial role in various applications, including materials processing, crystal growth, and casting.',
 'Refrences': [],
 'Answerable': True}

In [8]:
col_questions = client.get_or_create_collection('questions')
col_questions.add(ids='1', documents=str(not_found))
col_questions.get()

Add of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 1


{'ids': ['1', '0', '2', '3', '4', '5', '6', '7', '8', '9'],
 'embeddings': None,
 'metadatas': [None, None, None, None, None, None, None, None, None, None],
 'documents': ['[{\'Answer\': \'I apologize, but the provided text does not specify the essential materials used in the soldering process. It mainly discusses the overall process, safety aspects, and the importance of solderability. \', \'Refrences\': [], \'Answerable\': False, \'Question\': \'What are some of the essential materials used in the soldering process?\', \'Graph_Answer\': "Solder and flux are crucial materials. Solder is the material that melts and flows into the joint, while flux removes oxides and contaminants from the surfaces, improving the solder\'s wetting and flow."}, {\'Answer\': "The provided text focuses on various assembly variations and soldering processes used in electronics manufacturing, but it doesn\'t specifically mention how soldering contributes to assembly variations. The text describes different as

In [9]:
not_found

[]

In [10]:
question = base_questions[8]['Question']
print(question)
rag_results = col.query(query_texts=question,n_results=5)
rag_results

What are some entities related to soldering mentioned in the text?


NameError: name 'col' is not defined

In [121]:
res = answer_to_quesion("describe soldering metrics")
print(res['Answer'])

Soldering metrics are used to assess the quality of soldered joints in electronics assembly. This includes factors like solderability, which refers to the ability of surfaces to be wetted with molten solder and form smooth fillets, and the presence of non-wetting and de-wetting, which indicate incomplete wetting and retracting of solder, respectively. Inspections are conducted under magnification to observe wetting, penetration, and the presence of non-wetting or de-wetting, and non-wetting can be difficult to distinguish on plated through-hole boards. De-wetting resembles water on a greasy surface, with solder retracting into globules and ridges, leaving thin solder coatings between them.


In [88]:
col_questions = client.get_or_create_collection('questions')

for i,report in enumerate(communities_reports_content[:10]):
    base_questions = completion(create_base_question_prompt.format(doc=report))
    not_found = get_not_found_questions(base_questions)
    print(not_found)
    col_questions.add(ids=str(i), documents=str(not_found))


    



0 What is the central process in the community discussed in the text?
{'Answer': 'The central process discussed in the text is soldering, a process used in the electronics industry to bond components together, forming electrical connections. Soldering serves two functions: mechanical support, holding components together, and electrical support, forming required electrical connections.', 'Refrences': 'Soldering, in principle though not in practice, is a reasonably straightfor- ward process, used in the electronics industry to bond components to- gether, forming one or more electrical connections. From this description, it’s easy to see that soldering serves two functions: @ mechanical support — holding components of an assembly together @ electrical support — forming required electrical connections within a circuit.', 'Answerable': True}
1 What is the purpose of flux in soldering?
{'Answer': 'Flux plays a crucial role in soldering by cleaning the surface of the metal to be soldered, all

Insert of existing embedding ID: 1
Add of existing embedding ID: 1


{'Answer': 'Flux plays a crucial role in the success of soldering by cleaning the surfaces of metals to be joined, removing oxides and contaminants, thus allowing the solder to wet the surfaces effectively and form a strong bond.', 'Refrences': 'The effectiveness  of  a  flux  in  removing  oxide  and  contaminants  from  the   surface  of  metals  is  known  as  its  activity.', 'Answerable': True}
[{'Answer': "The text doesn't explicitly mention why fluxing the printed circuit board is crucial in soldering processes. It focuses on various fluxing methods and their applications in different soldering processes, but doesn't explain the importance of fluxing itself. Therefore, I can't provide an answer to your question based on the provided text.", 'Refrences': [], 'Answerable': False, 'Question': 'Why is fluxing the printed circuit board crucial in various soldering processes?', 'Graph_Answer': 'Fluxing is essential to clean the surface and ensure proper wetting and bonding of the sold

In [104]:

all_json = col_questions.get()['documents']
all_json

['[{\'Answer\': \'I apologize, but the provided text does not specify the essential materials used in the soldering process. It mainly discusses the overall process, safety aspects, and the importance of solderability. \', \'Refrences\': [], \'Answerable\': False, \'Question\': \'What are some of the essential materials used in the soldering process?\', \'Graph_Answer\': "Solder and flux are crucial materials. Solder is the material that melts and flows into the joint, while flux removes oxides and contaminants from the surfaces, improving the solder\'s wetting and flow."}, {\'Answer\': "The provided text focuses on various assembly variations and soldering processes used in electronics manufacturing, but it doesn\'t specifically mention how soldering contributes to assembly variations. The text describes different assembly methods for through-hole and surface mounted components, including CS (components before solder) and SC (solder before components) soldering processes.", \'Refrence

[{'Answer': 'I apologize, but the provided text does not specify the essential materials used in the soldering process. It mainly discusses the overall process, safety aspects, and the importance of solderability.',
  'Refrences': [],
  'Answerable': False,
  'Question': 'What are some of the essential materials used in the soldering process?',
  'Graph_Answer': "Solder and flux are crucial materials. Solder is the material that melts and flows into the joint, while flux removes oxides and contaminants from the surfaces, improving the solder's wetting and flow."},
 {'Answer': "The provided text focuses on various assembly variations and soldering processes used in electronics manufacturing, but it doesn't specifically mention how soldering contributes to assembly variations. The text describes different assembly methods for through-hole and surface mounted components, including CS (components before solder) and SC (solder before components) soldering processes.",
  'Refrences': [],
  '

In [115]:
import json_repair

all_questions =[]
for j in all_json:
    all_questions.extend( json_repair.loads(j) )

all_questions

[{'Answer': 'I apologize, but the provided text does not specify the essential materials used in the soldering process. It mainly discusses the overall process, safety aspects, and the importance of solderability.',
  'Refrences': [],
  'Answerable': False,
  'Question': 'What are some of the essential materials used in the soldering process?',
  'Graph_Answer': "Solder and flux are crucial materials. Solder is the material that melts and flows into the joint, while flux removes oxides and contaminants from the surfaces, improving the solder's wetting and flow."},
 {'Answer': "The provided text focuses on various assembly variations and soldering processes used in electronics manufacturing, but it doesn't specifically mention how soldering contributes to assembly variations. The text describes different assembly methods for through-hole and surface mounted components, including CS (components before solder) and SC (solder before components) soldering processes.",
  'Refrences': [],
  '